In [ ]:
!pip install optuna
!pip install catboost

In [ ]:
import os
import gc
import random
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,roc_auc_score
from catboost import CatBoostRegressor,  metrics, cv, Pool
import optuna
from optuna import trial


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['train']=1
test['train'] = 0

all_data = pd.concat([train,test])

from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors = 10)

all_data.loc[:,['song_duration_ms','acousticness','danceability','energy','instrumentalness','liveness',
               'loudness']] = knn_imputer.fit_transform(all_data.loc[:,['song_duration_ms','acousticness','danceability','energy','instrumentalness','liveness',
               'loudness']])

all_data['key'].fillna(all_data['key'].mode()[0], inplace=True)

train = all_data[all_data['train']==1]
test = all_data[all_data['train']==0]

train.drop('train',axis=1, inplace=True)
test.drop('train',axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
columns = [col for col in train.columns.to_list() if col not in ['id','song_popularity']]
data=train[columns]
target=train['song_popularity']

In [ ]:
NUM_CLASSES=target.nunique()

In [ ]:
model = CatBoostRegressor(silent=True)

grid = { 'depth':[3,1,2,6,4,5,7,8,9,10],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
             
}

randomized_search_result = model.randomized_search(grid,
                                                   X=data,
                                                   y=target,
                                                   plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 0.4787845833
bestIteration = 20

0:	loss: 0.4787846	best: 0.4787846 (0)	total: 5.49s	remaining: 49.4s

bestTest = 0.4779476828
bestIteration = 54

1:	loss: 0.4779477	best: 0.4779477 (1)	total: 7.87s	remaining: 31.5s

bestTest = 0.4776420786
bestIteration = 46

2:	loss: 0.4776421	best: 0.4776421 (2)	total: 12.3s	remaining: 28.8s

bestTest = 0.496964997
bestIteration = 999

3:	loss: 0.4969650	best: 0.4776421 (2)	total: 17.3s	remaining: 26s

bestTest = 0.4773173282
bestIteration = 54

4:	loss: 0.4773173	best: 0.4773173 (4)	total: 20.7s	remaining: 20.7s

bestTest = 0.4773491397
bestIteration = 417

5:	loss: 0.4773491	best: 0.4773173 (4)	total: 23.7s	remaining: 15.8s

bestTest = 0.478534623
bestIteration = 14

6:	loss: 0.4785346	best: 0.4773173 (4)	total: 34.8s	remaining: 14.9s

bestTest = 0.4773323144
bestIteration = 277

7:	loss: 0.4773323	best: 0.4773173 (4)	total: 38.4s	remaining: 9.59s

bestTest = 0.4775500897
bestIteration = 36

8:	loss: 0.4775501	best: 0.4773173 (4)	total

In [ ]:
model.get_params()

{'border_count': 20,
 'depth': 5,
 'l2_leaf_reg': 5,
 'learning_rate': 0.2,
 'loss_function': 'RMSE',
 'silent': True}

In [ ]:
params={'alpha': 0.4275932000556265,
 'colsample_bytree': 0.3303272480369658,
 'eval_metric': 'mlogloss',
 'lambda': 0.013685373489365672,
 'learning_rate': 0.08882012188858979,
 'max_depth': 17,
 'min_child_weight': 223,
 'num_class': 2,
 'objective': 'multi:softprob',
 'subsample': 0.5807927818586308,
 'n_estimators':5000,
 }

In [ ]:
from tqdm import tqdm
np.random.seed(64)
random.seed(64)
xtest = test.copy()

target = train['song_popularity'].values
FOLDS=10
kf = StratifiedKFold(n_splits = FOLDS, shuffle=True, random_state=64)    
        
oof = np.zeros((train.shape[0],))
predictions = 0
auc_value = []
for f, (train_index, test_index) in tqdm(enumerate(kf.split(X=train, y=target))):
        X_train, X_val = train.iloc[train_index][columns], train.iloc[test_index][columns]
        y_train, y_val = target[train_index], target[test_index]
        
        ss = StandardScaler()
        X_train[columns] = ss.fit_transform(X_train[columns])
        X_val[columns] = ss.transform(X_val[columns])
        xtest[columns] = ss.transform(xtest[columns])


        model = xgb.XGBClassifier(**params)
        
        model.fit(
            X_train[columns], 
            y_train,
            eval_set=[(X_val[columns], y_val)],
            early_stopping_rounds=30,
            verbose=100
        )
        
        oof_temp_pred = model.predict_proba(X_val[columns])[:,1]
        test_temp_pred = model.predict_proba(test[columns])[:,1]
        oof[test_index] = oof_temp_pred
        predictions += test_temp_pred/FOLDS
        auc = roc_auc_score(y_val, oof_temp_pred)
        auc_value.append(auc)
        print(f'\n FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_value)}')

print('\n')
print(np.mean(auc_value))

0it [00:00, ?it/s]

[0]	validation_0-mlogloss:0.686472
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[62]	validation_0-mlogloss:0.648044



1it [00:08,  8.88s/it]


 FOLD: 0 AUC: 0.5710054731912411 Mean AUC: 0.5710054731912411
[0]	validation_0-mlogloss:0.686285
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[48]	validation_0-mlogloss:0.648979



2it [00:16,  8.02s/it]

...


 FOLD: 7 AUC: 0.5653628101394514 Mean AUC: 0.5722134800557992
[0]	validation_0-mlogloss:0.686621
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[39]	validation_0-mlogloss:0.649764



9it [01:13,  7.57s/it]


 FOLD: 8 AUC: 0.567276476727332 Mean AUC: 0.5716649241304139
[0]	validation_0-mlogloss:0.686632
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[67]	validation_0-mlogloss:0.646199



10it [01:23,  8.31s/it]


 FOLD: 9 AUC: 0.5824911041822486 Mean AUC: 0.5727475421355974


0.5727475421355974


In [ ]:
params = {'border_count': 20,
 'depth': 5,
 'l2_leaf_reg': 5,
 'learning_rate': 0.2,
 'silent': True}

from tqdm import tqdm
np.random.seed(64)
random.seed(64)


target = train['song_popularity'].values
FOLDS=10
kf = StratifiedKFold(n_splits = FOLDS, shuffle=True, random_state=64)    
        
oof = np.zeros((train.shape[0],))
predictions = 0
auc_value = []
for f, (train_index, test_index) in tqdm(enumerate(kf.split(X=train, y=target))):
        X_train, X_val = train.iloc[train_index][columns], train.iloc[test_index][columns]
        y_train, y_val = target[train_index], target[test_index]
        
        model = CatBoostRegressor(**params)
        
        model.fit(
            X_train[columns], 
            y_train,
            eval_set=[(X_val[columns], y_val)],
            early_stopping_rounds=30,
            verbose=100
        )
        
        oof_temp_pred = model.predict(X_val[columns])
        test_temp_pred = model.predict(test[columns])
        oof[test_index] = oof_temp_pred
        predictions += test_temp_pred/FOLDS
        auc = roc_auc_score(y_val, oof_temp_pred)
        auc_value.append(auc)
        print(f'\n FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_value)}')

print('\n')
print(np.mean(auc_value))

0it [00:00, ?it/s]

0:	learn: 0.4805797	test: 0.4807233	best: 0.4807233 (0)	total: 3.45ms	remaining: 3.44s


1it [00:00,  2.43it/s]

Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.4777829775
bestIteration = 58

Shrink model to first 59 iterations.

 FOLD: 0 AUC: 0.5725879997873231 Mean AUC: 0.5725879997873231
0:	learn: 0.4805426	test: 0.4809005	best: 0.4809005 (0)	total: 4.11ms	remaining: 4.1s


2it [00:00,  2.52it/s]

...

Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.477692876
bestIteration = 63

Shrink model to first 64 iterations.

 FOLD: 7 AUC: 0.5713613488185857 Mean AUC: 0.5727288557353334
0:	learn: 0.4806495	test: 0.4808303	best: 0.4808303 (0)	total: 3.36ms	remaining: 3.36s


9it [00:03,  2.33it/s]

100:	learn: 0.4679717	test: 0.4783469	best: 0.4780009 (80)	total: 328ms	remaining: 2.92s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.4780009488
bestIteration = 80

Shrink model to first 81 iterations.

 FOLD: 8 AUC: 0.5702068082010967 Mean AUC: 0.5724486282315293
0:	learn: 0.4806691	test: 0.4806620	best: 0.4806620 (0)	total: 3.4ms	remaining: 3.4s


10it [00:04,  2.48it/s]

Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.4758130142
bestIteration = 64

Shrink model to first 65 iterations.

 FOLD: 9 AUC: 0.589964589410928 Mean AUC: 0.5742002243494693


0.5742002243494693


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission

,id,song_popularity
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
9995,9995,0
9996,9996,0
9997,9997,0
9998,9998,0


In [ ]:
submission['song_popularity']= model.predict(test)

In [ ]:
submission.to_csv('cat_boost_tuned.csv',index=False)